In [8]:
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # number of nodes in each layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #learning rate 
        self.lr = learningrate
        pass
    def train():
        pass
    def query():
        pass
    

In [9]:
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

#learning rate 
learning_rate = 0.3
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [10]:
import numpy
numpy.random.rand(3,3)-0.5

array([[ 0.48330831,  0.28058528,  0.29631851],
       [ 0.32461427, -0.06182669, -0.08732707],
       [-0.40374671,  0.08985069,  0.09397683]])

In [5]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline

class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #link weight matrices, wih and who
        #weights inside arrays are w_i_j, where link is from i to j in next layer
        #w11 w21
        #w12 w22 etc
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        #learning rate
        self.lr = learningrate
        
        #activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        pass

    def train(self, inputs_list, targets_list):
        #convert inputs list to 2darray
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        #output layer error(target- actual)
        output_errors = targets - final_outputs
        #hidden layer error is the output_error, split by weights and recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)

        #update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), 
                                      numpy.transpose(hidden_outputs))

        #update weights for link between input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), 
                                      numpy.transpose(inputs))
        pass

    def query(self, inputs_list):
        #convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T

        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate signals emerging from the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate signals emerging from the final output layer
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

#number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

learning_rate = 0.1

#object of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

#load mnist training data csv file into a list
training_data_file = open("mnist_dataset/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

#train the neural network

#epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    #go through all records in the training data set
    for record in training_data_list:
        all_values = record.split(',')
        #scale and shift inputs
        inputs = (numpy.array(all_values[1:], dtype=float) / 255.0 * 0.99) + 0.01
        #create the target output values(all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        #all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

#load the mnist test data CSV file into a list
test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

#test the neural network

#scorecard list for how well the network performs, initially empty
scorecard = []

#go through all the records in the test data set
for record in test_data_list:
    all_values = record.split(',')
    #correct answer is first value 
    correct_label = int(all_values[0])
    #scale and shift inputs
    inputs = (numpy.array(all_values[1:], dtype=float) / 255.0 * 0.99) + 0.01
    #query the network
    outputs = n.query(inputs)
    #index of highest value corresponds to label
    label = numpy.argmax(outputs)
    #append correct or incorrect to list
    if (label == correct_label):
        #if network's answer matches correct answer add 1 to scorecard
        scorecard.append(1)
    else:
        scorecard.append(0)
    pass
pass

#calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print("Performance = ", scorecard_array.sum() / scorecard_array.size)

Performance =  0.9677
